In [1]:
from pytket.extensions.nexus import Nexus
from datetime import datetime
from pytket.extensions.nexus import QuantinuumConfig
from pytket.extensions.nexus import NexusBackend
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.pauli import QubitPauliString

from pytket.circuit.display import render_circuit_jupyter as print_circ

import numpy as np
from pytket import Circuit
from pytket import OpType
from pytket.circuit import Op

from pytket.circuit.display import render_circuit_jupyter

In [351]:
num_qbit = 4
test_circuit1 = Circuit(num_qbit).X(1).CX(3,2).S(1).Y(3).Y(3)
def push_commute(test_circuit1:Circuit):
    new_circuit1 = Circuit(test_circuit1.n_qubits)
    
    flag=0
    
    list_gates_commute_X=[]
    list_gates_commute_X_next=[]

    flag1 = 0
    
    for g in test_circuit1:
        
        if g.op.type == OpType.X:
            g_X = g
            flag1=1
            
        if (flag1==1):
            if((not(g==g_X)) and (((g.op.get_name() == 'CX') and (g.args[0] == g_X.args[0])) or ((g.op.get_name() == 'CX') and (g.args[1] == g_X.args[0])) or  ((g.args[0] == g_X.args[0])))):
                list_gates_commute_X.append(g)
                #print(g)
                flag1=0

    if (len(list_gates_commute_X)>0):
        next_gate=list_gates_commute_X[0]
            
        if((next_gate.op.get_name() == 'CX') and (next_gate.args[0] == g_X.args[0])):
            print("CX1")
            new_circuit1.CX(parse_index(str(next_gate.args[0])),parse_index(str(next_gate.args[1]))).X(parse_index(str(next_gate.args[0]))).X(parse_index(str(next_gate.args[1])))
        elif ((next_gate.op.get_name() == 'CX') and (next_gate.args[1] == g_X.args[0])):
            new_circuit1.CX(parse_index(str(next_gate.args[0])),parse_index(str(next_gate.args[1]))).X(parse_index(str(next_gate.args[1])))
            print("CX2")
        elif (next_gate.args[0] == g_X.args[0]):
            new_circuit1.add_gate(next_gate.op, next_gate.qubits).add_gate(next_gate.op, next_gate.qubits).add_gate(g_X.op, g_X.qubits).add_gate(next_gate.op.dagger, next_gate.qubits)
            print("Single")
        else:
            new_circuit1.add_gate(g.op, g.qubits).add_gate(g_X.op, g_X.qubits)
            flag=0

    else:
         new_circuit1.add_gate(g_X.op, g_X.qubits)
        

    
    for g in test_circuit1:
        if (g not in list_gates_commute_X) and (not(g_X==g)):
            print(g)
            new_circuit1.add_gate(g.op, g.qubits)



    return new_circuit1 

In [352]:
new_circuit1 = push_commute(test_circuit1) 
new_circuit2 = push_commute(new_circuit1) 
print_circ(test_circuit1)

Single
CX q[3], q[2];
Y q[3];
Y q[3];
Single
S q[1];
CX q[3], q[2];
S q[1];
Y q[3];
Y q[3];


In [353]:
print_circ(new_circuit1)
print_circ(new_circuit2)

In [239]:
def parse_index(s):
    # Find the index within square brackets
    start_index = s.find('[')
    end_index = s.find(']')

#Extract the substring containing the index
    index_str = s[start_index + 1:end_index]

#Convert the index string to an integer
    index = int(index_str)

    return index

5


In [229]:
new_circuit1 = push_commute(test_circuit1) 
print_circ(test_circuit1)
print_circ(new_circuit1)

S q[0];
X q[1];
